In [116]:
%reload_ext autoreload
%autoreload 2


In [117]:
import numpy as np
from scipy import stats
import pandas as pd
import math
import os

import sys

from Classes import DataO
from Classes import MapO


In [118]:
datapath="../3_Raw_Data/"
resultpath="../6_Results/2707_Maps/"

In [7]:
'TODO:plot spatial distribution of values, 3D histogram?'

'TODO:plot spatial distribution of values, 3D histogram?'

In [119]:
#### RUNNING ANALYSIS ####
#** Generating the threshold table ***

thresh_df=pd.DataFrame(
    columns=["Maxtemp",
            "Mintemp",
            "Avgtemp",
            "Rain",
            "Wind",
            "Snow"],
    index= ["Thermoelectric",
            "Hydroelectric",
            "Wind",
            "Solar PV",
            "Transformers",
            "Poles",
            "Overhead Lines",
            "Demand"]
)

thresh_df['Maxtemp']=[22,35,35,35,35,35,35,35]
thresh_df['Mintemp']=[3,0,0,0,0,0,0,0]
thresh_df['Avgtemp']=[20,25,25,25,25,25,25,25]
thresh_df['Rain']=[5,50,50,50,50,50,50,50]
thresh_df['Wind']=[20,30,30,30,30,30,30,30]
thresh_df['Snow']=[1,50,50,50,50,50,50,50]

'TESTING'
print(thresh_df)

                Maxtemp  Mintemp  Avgtemp  Rain  Wind  Snow
Thermoelectric       22        3       20     5    20     1
Hydroelectric        35        0       25    50    30    50
Wind                 35        0       25    50    30    50
Solar PV             35        0       25    50    30    50
Transformers         35        0       25    50    30    50
Poles                35        0       25    50    30    50
Overhead Lines       35        0       25    50    30    50
Demand               35        0       25    50    30    50


In [120]:
### LOAD ALL WEAtHER VARIABLE DATA ###

spatial_res=5
time_res='monthly'
#creating result directory
if not os.path.exists(resultpath):
    os.mkdir(resultpath)

Maxtemp=DataO.DataObject(title="5_Monthly_Maxtemp_2021_2040_UK",
                        varname="Maxtemp",sres=spatial_res,tres=time_res,
                        fdname="5_Monthly_Maxtemp_2021_2040_UK",
                        fdpath="../3_Raw_Data/5_Monthly_Maxtemp_2021_2040_UK",
                        respath=resultpath)


Avgtemp=DataO.DataObject(title="5_Monthly_Avgtemp_2021_2040_UK",
                        varname="Avgtemp",sres=spatial_res,tres=time_res,
                        fdname="5_Monthly_Avgtemp_2021_2040_UK",
                        fdpath="../3_Raw_Data/5_Monthly_Avgtemp_2021_2040_UK",
                        respath=resultpath)

Mintemp=DataO.DataObject(title="5_Monthly_Mintemp_2021_2040_UK",
                        varname="Mintemp",sres=spatial_res,tres=time_res,
                        fdname="5_Monthly_Mintemp_2021_2040_UK",
                        fdpath="../3_Raw_Data/5_Monthly_Mintemp_2021_2040_UK",
                        respath=resultpath)

Rain=DataO.DataObject(title="5_Monthly_Rain_2021_2040_UK",
                        varname="Rain",sres=spatial_res,tres=time_res,
                        fdname="5_Monthly_Rain_2021_2040_UK",
                        fdpath="../3_Raw_Data/5_Monthly_Rain_2021_2040_UK",
                        respath=resultpath)

Snow=DataO.DataObject(title="5_Monthly_Snow_2021_2040_UK",
                        varname="Snow",sres=spatial_res,tres=time_res,
                        fdname="5_Monthly_Snow_2021_2040_UK",
                        fdpath="../3_Raw_Data/5_Monthly_Snow_2021_2040_UK",
                        respath=resultpath)

WeatherVariables=[Maxtemp,Mintemp,Avgtemp,Rain,Snow]

'TESTING'
for object in WeatherVariables:
    assert object.untreated_array.shape == (12,239,243,153), 'Loaded data array does not have expected shape'


In [121]:
### RUNNING STAT TESTS ###

Maxtemp.run_stats(KStest=False,stats=True,tp_90=True)
Mintemp.run_stats(KStest=False,stats=True,tp_90=True)
Avgtemp.run_stats(KStest=False,stats=True,tp_90=True)
Rain.run_stats(KStest=False,stats=True,tp_90=True)
Snow.run_stats(KStest=False,stats=True,tp_90=True)


In [122]:
'TODO: update with correct row as component'

### SETTING THRESHOLDS FOR EACH VARIABLE ###
for variable in WeatherVariables:
    threshold=thresh_df[variable.varname]["Thermoelectric"]
    variable.set_threshold(threshold)


'Testing'
for variable in WeatherVariables:
    assert variable.threshold==thresh_df[variable.varname]["Thermoelectric"]


In [124]:
### COUNTING EXCESS DAYS ###

Maxtemp.counter()
Mintemp.counter()
Avgtemp.counter()
Rain.counter()
Snow.counter()


In [126]:
### CREATING MAP ###

Map_5_Monthly_2021_2040=MapO.MapObject("Maxtemp - Excess days",Maxtemp,respath=resultpath)

for variable in WeatherVariables:
    if not (variable.title in Map_5_Monthly_2021_2040.dnames):
        Map_5_Monthly_2021_2040.addDataObject(variable)

Map_5_Monthly_2021_2040.build_props()
Map_5_Monthly_2021_2040.geojson_write('allwv')